In [ ]:
'''BARCLAYS OPTION TRADING STRAT'''
#This strat aims to exploit the gap between implied option vol and historical vol (volatility risk premium)
#Retail traders in the past years have created huge liquidity for short-dated calls on large cap stocks
#We build a strat around this that basically buys undervalued calls and sells overvalued ones
#The expiration date we select is going to be 2 weeks, as this is where most liquidiy is concentrated

'BARCLAYS OPTION TRADING STRAT'

In [170]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from polygon import RESTClient
import dotenv
import os
import pandas as pd
import warnings
from datetime import datetime, timedelta

warnings.filterwarnings('ignore')

dotenv.load_dotenv()
POLYGON_API = os.getenv("POLYGON_API")

client = RESTClient(api_key=POLYGON_API)

In [174]:
'''Taking data from spy stocks to build the strat around'''
#We parse the wikipedia page for spy data

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
tickers = sp500['Symbol'].unique().tolist()

data = yf.download(tickers, period='1y')
data = data.stack()
data.index.names = ['date', 'ticker']
data.columns = data.columns.str.lower()

[*********************100%***********************]  503 of 503 completed


In [175]:
'''Well now filter by dollar volume to only opearte with the stocks which fit our liquidity constraints'''
#We will compute the mean dollar volume from the past year to filter liquid stocks
#Further, we selecct large cap stocks, as retail trader liquidity is more likely concentrated here

latest_date = data.index.get_level_values('date').max() #we use this below for option vol

data['dollarvol'] = (data['close'] * data['volume']) / 1e6
data = data.groupby('ticker').mean() #finding mean dollar vol over past year

data = data.sort_values(by='dollarvol', ascending = False)
data = data.head(100) #top 100 liquid stocks

In [ ]:
'''Lets try to build a options volume column in the dataframe'''
#I tweaked with the weeks value and found the most liquidity to be in 2 weeks expriration date options, so we go with that
#This is an incredibly slow process, as yfinance doesn't give option volume directly and we have to find a workaround
#We now filter top 50 stocks with highest option volume

'''BUG => Too slow when trying to first filter by option volume, so did that later'''

data['option_volume'] = 0

latest_tickers = data.index.get_level_values('ticker')

count = 1

for tckr in latest_tickers:
    expiration_date = (latest_date + timedelta(weeks=2)).strftime('%Y-%m-%d')

    tk = yf.Ticker(tckr)
    opt_chain = tk.option_chain(expiration_date)

    option_volume = opt_chain.calls['volume'].sum() + opt_chain.puts['volume'].sum()

    data.loc[tckr, 'option_volume'] = option_volume

    print("Ticker ", count, " done")
    count += 1

Ticker  1  done
Ticker  2  done
Ticker  3  done
Ticker  4  done
Ticker  5  done
Ticker  6  done
Ticker  7  done
Ticker  8  done
Ticker  9  done
Ticker  10  done
Ticker  11  done
Ticker  12  done
Ticker  13  done
Ticker  14  done
Ticker  15  done
Ticker  16  done
Ticker  17  done
Ticker  18  done
Ticker  19  done
Ticker  20  done
Ticker  21  done
Ticker  22  done
Ticker  23  done
Ticker  24  done
Ticker  25  done
Ticker  26  done
Ticker  27  done
Ticker  28  done
Ticker  29  done
Ticker  30  done
Ticker  31  done
Ticker  32  done
Ticker  33  done
Ticker  34  done
Ticker  35  done
Ticker  36  done
Ticker  37  done
Ticker  38  done
Ticker  39  done
Ticker  40  done
Ticker  41  done
Ticker  42  done
Ticker  43  done
Ticker  44  done
Ticker  45  done
Ticker  46  done
Ticker  47  done
Ticker  48  done
Ticker  49  done
Ticker  50  done
Ticker  51  done
Ticker  52  done
Ticker  53  done
Ticker  54  done
Ticker  55  done
Ticker  56  done
Ticker  57  done
Ticker  58  done
Ticker  59  done
Ticker

In [184]:
'''Checking how different max_dollar_volume and max_option_volume tickers are'''

max_dollar_vol = data.head(50)
data = data.sort_values(by='option_volume', ascending = False)

max_option_vol = data.head(50)

max_dvol= set(max_dollar_vol.index.to_list())
max_ovol = set(max_option_vol.index.to_list())

max_dvol.difference(max_ovol)
max_ovol.difference(max_dvol)

set()

['NVDA',
 'TSLA',
 'AAPL',
 'MSFT',
 'META',
 'AMZN',
 'AMD',
 'AVGO',
 'GOOGL',
 'SMCI',
 'PLTR',
 'GOOG',
 'LLY',
 'NFLX',
 'MU',
 'UNH',
 'V',
 'JPM',
 'CRM',
 'INTC',
 'XOM',
 'COST',
 'BRK-B',
 'ADBE',
 'CRWD',
 'BAC',
 'QCOM',
 'BA',
 'UBER',
 'ORCL',
 'WMT',
 'HD',
 'DELL',
 'MA',
 'AMAT',
 'NOW',
 'MRK',
 'JNJ',
 'GS',
 'CVX',
 'PG',
 'WFC',
 'TXN',
 'PANW',
 'DIS',
 'BKNG',
 'ABBV',
 'PFE',
 'CSCO',
 'NKE']

In [ ]:
'''Now we download the data for the 50 stocks we selected'''

tickers = " ".join(data.index.tolist())

options_data = yf.download(tickers, period="1y")
options_data = options_data.stack()
options_data.index.names = ['date', 'ticker']
options_data.columns = options_data.columns.str.lower()

[*********************100%***********************]  100 of 100 completed


In [112]:
for tck in data.index.tolist():
    dta = yf.Ticker(tck)
    tmp = dta.option_chain(dta.options[0]).calls
    break

tmp

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,NVDA250314C00050000,2025-03-07 20:52:50+00:00,50.0,62.83,62.35,63.10,-1.809998,-2.800120,142,148,2.265629,True,REGULAR,USD
1,NVDA250314C00055000,2025-03-03 16:48:26+00:00,55.0,64.07,57.05,58.35,0.000000,0.000000,10,90,1.781251,True,REGULAR,USD
2,NVDA250314C00060000,2025-03-07 20:53:52+00:00,60.0,52.00,52.05,53.35,-2.349998,-4.323824,39,29,1.562502,True,REGULAR,USD
3,NVDA250314C00065000,2025-03-07 20:32:00+00:00,65.0,47.50,47.10,48.30,1.700001,3.711792,185,299,1.375003,True,REGULAR,USD
4,NVDA250314C00070000,2025-03-07 20:00:45+00:00,70.0,41.85,42.30,43.60,0.680000,1.651689,112,128,1.828126,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,NVDA250314C00220000,2025-02-27 14:30:01+00:00,220.0,0.01,0.00,0.01,-0.010000,-50.000000,1,112,1.500002,False,REGULAR,USD
73,NVDA250314C00225000,2025-03-07 20:16:27+00:00,225.0,0.01,0.00,0.01,0.000000,0.000000,3,194,1.531252,False,REGULAR,USD
74,NVDA250314C00230000,2025-03-04 19:13:33+00:00,230.0,0.01,0.00,0.01,0.000000,0.000000,1,843,1.562502,False,REGULAR,USD
75,NVDA250314C00235000,2025-02-28 16:07:55+00:00,235.0,0.01,0.00,0.01,0.000000,0.000000,10,305,1.625002,False,REGULAR,USD
